In [1]:
!nvidia-smi

Thu Jul 15 08:15:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    62W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7f39910364d0>)

In [3]:
# 可用gpu数量
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):
  if torch.cuda.device_count() >= i + 1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus():
  devices = [
    torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
  return devices if devices else torch.device('cpu')

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 张量与GPU

In [5]:
# 张量默认创建在cpu
X = torch.tensor([1, 2, 3])
X.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu(0))
Y = torch.rand(2, 3, device=try_gpu(0))
X, Y

(tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'), tensor([[0.4634, 0.0362, 0.6177],
         [0.0261, 0.0538, 0.7408]], device='cuda:0'))

In [7]:
Z = X + Y
Z

tensor([[1.4634, 1.0362, 1.6177],
        [1.0261, 1.0538, 1.7408]], device='cuda:0')

## 神经网络与GPU

In [8]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)

tensor([[-0.1423],
        [-0.1423]], device='cuda:0', grad_fn=<AddmmBackward>)

In [9]:
net[0].weight.data.device

device(type='cuda', index=0)